In [1]:
import os
import numpy as np
import polyscope as ps
from skimage import measure


def load_off_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the vertices and faces from the OFF file
    num_vertices, num_faces, _ = map(int, lines[1].split())

    vertices = np.array([list(map(float, line.split())) for line in lines[2:2 + num_vertices]])
    faces = np.array([list(map(int, line.split()))[1:] for line in lines[2 + num_vertices:]])

    return vertices, faces

def compute_RBF_weights(inputPoints, inputNormals, RBFFunction, epsilon, RBFCentreIndices=[], useOffPoints=True,
                        sparsify=False, l=-1):

    ##RBF computation is done in this function
    w=[]  #RBF weights
    RBFCentres=[] #RBF centres
    a=[] #polynomial coefficients (for Section 2)
    return w, RBFCentres, a


def evaluate_RBF(xyz, centres, RBFFunction, w, l=-1, a=[]):
    values = np.zeros(xyz.shape[0])

    ###Complate RBF evaluation here
    return values



In [2]:



ps.init()

inputPointNormals, _ = load_off_file(os.path.join('..', 'data', 'bunny-500.off'))
inputPoints = inputPointNormals[:, 0:3]
inputNormals = inputPointNormals[:, 3:6]

# normalizing point cloud to be centered on [0,0,0] and between [-0.9, 0.9]
inputPoints -= np.mean(inputPoints, axis=0)
min_coords = np.min(inputPoints, axis=0)
max_coords = np.max(inputPoints, axis=0)
scale_factor = 0.9 / np.max(np.abs(inputPoints))
inputPoints = inputPoints * scale_factor

ps_cloud = ps.register_point_cloud("Input points", inputPoints)
ps_cloud.add_vector_quantity("Input Normals", inputNormals)


# Parameters
gridExtent = 1 #the dimensions of the evaluation grid for marching cubes
res = 50 #the resolution of the grid (number of nodes)

# Generating and registering the grid
gridDims = (res, res, res)
bound_low = (-gridExtent, -gridExtent, -gridExtent)
bound_high = (gridExtent, gridExtent, gridExtent)
ps_grid = ps.register_volume_grid("Sampled Grid", gridDims, bound_low, bound_high)

X, Y, Z = np.meshgrid(np.linspace(-gridExtent, gridExtent, res),
                        np.linspace(-gridExtent, gridExtent, res),
                        np.linspace(-gridExtent, gridExtent, res), indexing='ij')

#the list of points to be fed into evaluate_RBF
xyz = np.column_stack((X.flatten(), Y.flatten(), Z.flatten()))

##########################
## you code of computation and evaluation goes here
##
##
RBFValues = xyz[:,0]**2+xyz[:,1]**2+xyz[:,2]**2-0.5 #stub sphere
##
##
#########################

#fitting to grid shape again
RBFValues = np.reshape(RBFValues, X.shape)

# Registering the grid representing the implicit function
ps_grid.add_scalar_quantity("Implicit Function", RBFValues, defined_on='nodes',
                            datatype="standard", enabled=True)

# Computing marching cubes and realigning result to sit on point cloud exactly
vertices, faces, _, _ = measure.marching_cubes(RBFValues, spacing=(
    2.0 * gridExtent / float(res - 1), 2.0 * gridExtent / float(res - 1), 2.0 * gridExtent / float(res - 1)),
                                                level=0.0)
vertices -= gridExtent
ps.register_surface_mesh("Marching-Cubes Surface", vertices, faces)

ps.show()
